In [1]:
import pandas as pd

import json
import glob
import os
import re

In [2]:
base = "/amd.home/home/i11/zeitz/experiments/chpot_old/"
paths = glob.glob(base + "applications/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'CH Potentials Query'])

queries = queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in ['CH Potentials Query', 'CH Potentials TD Query']]))

In [18]:
dijkstra_queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'Dijkstra Query'])

dijkstra_queries = dijkstra_queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in ['Dijkstra Query', 'TD Dijkstra Query']]))

In [9]:
runtime_pattern = re.compile(".*Needed (\\d+)musec\\..*")

def parse_contraction_output(path):
    stats = { 'preprocessing_running_time_s': 0.0 }

    with open(path, 'r') as f:
        for line in f:
            if not 'graph' in stats:
                stats['graph'] = line.strip()
            else:
                match = runtime_pattern.match(line)
                if match:
                    stats['preprocessing_running_time_s'] += int(match[1]) / 1000000

    return stats

preprocessing = pd.DataFrame.from_records([parse_contraction_output(path) for path in glob.glob(base + "preprocessing/*.out")])

In [7]:
queries['affected'] = queries['lower_bound'] != queries['result']
queries['increase'] = queries['result'] / queries['lower_bound']

In [23]:
table = queries.groupby(['graph', 'exp']).agg(
    running_time_ms=('running_time_ms', 'mean'), 
    num_pot_evals=('num_pot_evals', 'mean'), 
    num_relaxed_arcs=('num_relaxed_arcs', 'mean'),
    increase=('increase', 'mean'),
    affected=('affected', 'sum'),
    size=('affected', 'count'))
table['affected'] = table['affected'] * 100 / table['size']
table = table[table.columns[0:-1]]
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000
table['num_pot_evals'] = table['num_pot_evals'] / 1000
table = table.join(preprocessing.groupby('graph').mean())
table = table.join(dijkstra_queries.groupby(['graph', 'exp']).agg(dijkstra_running_time_ms=('running_time_ms', 'mean')))
#table = table.round(1)
print(table.to_latex())
table

\begin{tabular}{llrrrrr}
\toprule
                                         &            &  running\_time\_ms &  num\_pot\_evals &  num\_relaxed\_arcs &  increase &  affected \\
graph & exp &                  &                &                   &           &           \\
\midrule
/algoDaten/graphs/cleaned\_td\_road\_data/de/day/dido/ & chpot\_td &         4.393615 &   1.874865e+04 &         30.077662 &  1.030244 &     98.23 \\
/algoDaten/graphs/cleaned\_td\_road\_data/ptv17-eur-car/day/di/ & chpot\_td &        87.384608 &   2.490319e+05 &        500.442628 &  1.038706 &     99.46 \\
/algoDaten/zeitz/roadgraphs/europe/ & fastest\_times\_10 &         0.953474 &   3.542860e+03 &          2.856519 &  1.000004 &     16.61 \\
                                         & fastest\_times\_2 &         0.957054 &   3.538288e+03 &          2.852366 &  1.000000 &     16.61 \\
                                         & random\_times\_10 &         0.971330 &   3.609995e+03 &          2.908221 &  1.0000

running_time_ms  \
graph                                              exp                                 
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 4.393615   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                87.384608   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10         0.953474   
                                                   fastest_times_2          0.957054   
                                                   random_times_10          0.971330   
                                                   random_times_2           0.960500   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live             102.182746   
                                                   no_highways            322.861485   
                                                   no_tunnels              29.063405   

                                                                     num_pot_evals  \
graph                                              exp                               
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td           1.874865e+04   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td           2.490319e+05   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10   3.542860e+03   
                                                   fastest_times_2    3.538288e+03   
                                                   random_times_10    3.609995e+03   
                                                   random_times_2     3.562128e+03   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live         5.274343e+05   
                                                   no_highways        1.547422e+06   
                                                   no_tunnels         1.510985e+05   

                                                                     num_relaxed_arcs  \
graph                                              exp                                  
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 30.077662   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                500.442628   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10          2.856519   
                                                   fastest_times_2           2.852366   
                                                   random_times_10           2.908221   
                                                   random_times_2            2.864245   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              801.182286   
                                                   no_highways            2464.919733   
                                                   no_tunnels              224.131654   

                                                                     increase  \
graph                                              exp                          
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td          1.030244   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td          1.038706   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10  1.000004   
                                                   fastest_times_2   1.000000   
                                                   random_times_10   1.000056   
                                                   random_times_2    1.000007   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live        1.132316   
                                                   no_highways       8.238283   
                                                   no_tunnels        1.626779   

                                                                     affected  
graph                                              exp                         
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td             98.23  
/algoDaten/graphs/cleaned_td_road_data/pt